# Initialize

In [95]:
%pip install PyCryptoDome
%pip install requests

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\ballb\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Users\ballb\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [97]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [98]:
from function import *

# Set_1

Challenge 1

In [3]:
text = '49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d'
ans = 'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'

baseInt= int(text, 16)
print(baseInt)

bytes_text = bytes.fromhex(text)
print(bytes_text)

result = base64.b64encode(bytes_text)
print(result)

11259432467145572969189485457381052543241507215288737798329079056359121649591228422793827173000297562297701340508013
b"I'm killing your brain like a poisonous mushroom"
b'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'


Challenge 2

In [4]:
buffer1 = '1c0111001f010100061a024b53535009181c'
buffer2 = '686974207468652062756c6c277320657965'
ans = '746865206b696420646f6e277420706c6179'

hex1 = bytes.fromhex(buffer1)
hex2 = bytes.fromhex(buffer2)
print(hex1)
print(hex2)

result = ''
for i in range(len(hex1)):
    result += str(hex(hex1[i] ^ hex2[i]))[2:]

print(result)
print(result == ans)

b'\x1c\x01\x11\x00\x1f\x01\x01\x00\x06\x1a\x02KSSP\t\x18\x1c'
b"hit the bull's eye"
746865206b696420646f6e277420706c6179
True


Challenge 3

In [5]:
text = '1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'
print(try_decrypt(text))

NameError: name 'try_decrypt' is not defined

Challenge 4

In [141]:
text_list = get_txt("https://cryptopals.com/static/challenge-data/4.txt")

In [7]:
decrypt_list = []
for text in text_list:
  decrypt_list.append(try_decrypt(text))
result = sorted(decrypt_list, key=lambda x:x[0], reverse=True)
print(*result[:10], sep="\n")

(219.54000000000002, 'Now that the party is jumping\n', 53)
(113.88999999999997, '[$LrLi;hsDbv!5 j$OaEfG=>rKDl=S', 109)
(113.14999999999999, 'wpUs{gnns F8b-aT2*zRAh{Gj>xuED', 98)
(109.14, "iu`'ern.&nZeM,MjT\x0b:FD~i5+OyEmL", 113)
(105.78999999999999, 'Sw6XSo$^uWyP$C{iUCns3e?E` /]GZ', 108)
(90.35000000000001, 'T2XNi-]1rTIYbP/>2`o%%]T5JonmqA', 99)
(70.12000000000002, ']$4OFs>\\YQ#\nK9y?IT4PeNWTxFqc0\r', 103)
(70.01000000000002, '\\`)]^o}me|~#Am~t[dbF.tq\x0bmAWM|~', 114)
(0, "\x0e6GèY-5QJ\x08\x12CX%6í=æs@Y\x00\x1e?S\\æ'\x102", 0)
(0, '3K\x04\x1dá$÷<\x18\x01\x1aPæ\x08\tzÃ\x08ìîP\x137ì>\x10\x08T \x1d', 0)


Challenge 5

In [8]:
text = '''Burning 'em, if you ain't quick and nimble
I go crazy when I hear a cymbal'''

key = 'ICE'
print(encrypt(text, key).hex())


0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f


Challenge 6

In [9]:
text_list = get_txt("https://cryptopals.com/static/challenge-data/6.txt")

In [10]:
text = "".join(text_list)
text = base64.b64decode(text)

print(text)
print(len(text))

b'\x1dB\x1fM\x0b\x0f\x02\x1fO\x13N<\x1aie\x1fI\x1c\x0eN\x13\x01\x0b\x07N\x1b\x01\x16E6\x00\x1e\x01Id T\x1d\x1dC3SNeR\x06\x00GT\x1c\rEM\x07\x04\x0cS\x12<\x0c\x1e\x08I\x1a\t\x11O\x14L!\x1aG+\x00\x05\x1dGY\x11\x04\t\x00d&\x07S\x007\x16\x06\x0c\x1a\x17A\x1d\x01RT0_\x00 \x13\n\x05GO\x12H\x08ENe>\x16\t8E\x06\x05\x08\x1aF\x07O\x1fYx~jb6\x0c\x1d\x0fA\rH\x06U\x1a\x1b\x00\x1dBt\x04\x1e\x01I\x1a\t\x11\x02Rz\x7fI\x00H:\x00\x1a\x13I\x1aOEH\x0f\x1d\rS\x04:\x01R\x19\x01\x0bA\x13\x06\x00L1_Sb\x15\x06\x07\t\x07T\x0b\x17A\x14\x16Iy35\x0b\x1b\x01\x05\x0fF\x07O\x1dNxNH\'R\x04\x07\x0cEXH\x08A\x00O T\x08t\x0b\x1d\x19I\x02\x00\x0e\x16\\\x00R0ie\x1fI\x02\x02T\x00\x01\x0b\x07N\x02\x10S\x01&\x10\x15M\x02\x07\x02\x1fO\x1bNx0i6R\n\x01\tT\x06\x07\tSN\x02\x10S\x08;\x10\x06\x05I\x0f\x0f\x10O;\x00:_G+\x1cId3OT\x02\x10S\x1aO\x05\x16\x11t\x0c\x06M\x0f\x02\x0e\x03CRL=N\x00/\x0bI\r\x08N\x17\r\x15T\x1dO\x0e\x1cE^(\x0bM\x19\x01\x12\x07\nUSxNOb\x06\x01\x0bGS\x1d\x0c\x00\x00\x17\n\x05\x1f\x0c:B^M.\x01A"\x0e\x1cI4VAb5\x06OG*~

In [11]:
test_Normalized = defaultdict(list)
Normalized_list = dict()
for keysize in range(2, 41):
  block_list = []
  for i in range(int(len(text)/keysize) + 1):
    block_list += [text[keysize*i:keysize*(i+1)]]
  
  distance_list = []
  for i in range(len(block_list) - 1):
    distance_list += [hamming_distance(block_list[i],block_list[i+1])]
  Normalized_list[keysize] = sum(distance_list)/len(distance_list)/keysize

  
  for i in range(2, len(distance_list) + 1):
    test_distance_list = distance_list[:i]
    test_Normalized[i] += [(keysize, sum(test_distance_list)/len(test_distance_list)/keysize)]

max_chunk = int(len(text) / 40) + 1
for chunk, normalized_list in test_Normalized.items():
  if chunk > max_chunk:
    break
  print(f"for using {chunk} chunk, best 5 scores are : {sorted(normalized_list, key=lambda x:x[1])[:5]}")

print(*sorted(Normalized_list.items(), key=lambda x:x[1])[:10], sep='\n')
candidate_keysize_list = [x[0] for x in sorted(Normalized_list.items(), key=lambda x:x[1])[:10]]

transpose_block_dict = dict()
for keysize in candidate_keysize_list:
  block_list = []
  for i in range(keysize):
    block_list += [text[i::keysize]]
  transpose_block_dict[keysize] = block_list

print(transpose_block_dict)
for keysize, transpose_block_list in transpose_block_dict.items():
  decrypt_list = []
  for index, transpose_block in enumerate(transpose_block_list):
    score, _, key = try_decrypt(transpose_block)
    decrypt_list.append((index, (score, key)))
  #print(decrypt_list)

  key = "".join([chr(x[1][1]) for x in decrypt_list])
  if all([x in string.printable for x in key]):
    print(f"key: {key}")

for using 2 chunk, best 5 scores are : [(2, 2.25), (5, 2.3), (3, 2.3333333333333335), (18, 2.8333333333333335), (20, 2.85)]
for using 3 chunk, best 5 scores are : [(2, 2.0), (3, 2.6666666666666665), (29, 2.793103448275862), (5, 2.8), (18, 2.9444444444444446)]
for using 4 chunk, best 5 scores are : [(2, 2.25), (5, 2.6), (29, 2.8189655172413794), (3, 2.8333333333333335), (18, 3.0555555555555554)]
for using 5 chunk, best 5 scores are : [(2, 2.4), (5, 2.52), (3, 2.8000000000000003), (29, 2.8344827586206898), (31, 3.0258064516129033)]
for using 6 chunk, best 5 scores are : [(2, 2.5833333333333335), (5, 2.7), (3, 2.8333333333333335), (29, 2.8563218390804597), (18, 3.0185185185185186)]
for using 7 chunk, best 5 scores are : [(5, 2.7142857142857144), (29, 2.8226600985221677), (3, 2.857142857142857), (15, 2.99047619047619), (18, 3.0238095238095237)]
for using 8 chunk, best 5 scores are : [(5, 2.775), (29, 2.8189655172413794), (3, 2.8333333333333335), (2, 3.0), (15, 3.0833333333333335)]
for usin

Chellange 7

In [12]:
text_list = get_txt("https://cryptopals.com/static/challenge-data/7.txt")
text = ''.join(text_list)
text = base64.b64decode(text)
key = b'YELLOW SUBMARINE'

In [13]:
print(decrypt_ECB(text, key))

b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \nI can take 

Challenge 8

In [14]:
text_list = get_txt("https://cryptopals.com/static/challenge-data/8.txt")

In [15]:
print(len(text_list))

205


In [16]:
target_index = 0
for index, text in enumerate(text_list):
  text = bytes.fromhex(text)
  if len(text) < 1:
    continue
  for i in range(10):
    for j in range(i + 1, 10):
      if text[i*16: (i+1)*16] == text[j*16: (j+1)*16]:
        print(index, text)
        target_index = index
        break

print(text_list[target_index])

132 b'\xd8\x80a\x97@\xa8\xa1\x9bx@\xa8\xa3\x1c\x81\n=\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\xe2\xdd\x05/kd\x1d\xbf\x9d\x11\xb04\x85B\xbbW\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\x94u\xc9\xdf\xdb\xc1\xd4e\x97\x94\x9d\x9c~\x82\xbfZ\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\x97\xa9>\xab\x8dj\xec\xd5fH\x91Tx\x9ak\x03\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\xd4\x03\x18\x0c\x98\xc8\xf6\xdb\x1f*?\x9c@@\xde\xb0\xabQ\xb2\x993\xf2\xc1#\xc5\x83\x86\xb0o\xba\x18j'
132 b'\xd8\x80a\x97@\xa8\xa1\x9bx@\xa8\xa3\x1c\x81\n=\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\xe2\xdd\x05/kd\x1d\xbf\x9d\x11\xb04\x85B\xbbW\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\x94u\xc9\xdf\xdb\xc1\xd4e\x97\x94\x9d\x9c~\x82\xbfZ\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\x97\xa9>\xab\x8dj\xec\xd5fH\x91Tx\x9ak\x03\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\xd4\x03\x18\x0c\x98\xc8\xf6\xdb\x1f*?\x9c@@\xde\xb0\xabQ\xb2\x993\xf2\xc1#\xc5\x83\x86\xb0o\xba\x18j'
132 b'\xd8\x80a\x97@\xa8\xa1\x

# Set_2

Challenge 9

In [17]:
padding_PKCS7(b"YELLOW SUBMARINE", 20)

b'YELLOW SUBMARINE\x04\x04\x04\x04'

Challenge 10

In [18]:
plain_text_list = get_txt("https://cryptopals.com/static/challenge-data/10.txt")
plain_text = ''.join(text_list)
key = b'YELLOW SUBMARINE'
print(plain_text)

...


In [19]:
block_size = 16
initialization_vector = b'\x00' * block_size

print(decrypt_CBC(plain_text.encode(), key, block_size, initialization_vector))

b'\x9czM\x11"\x0b$0\ra\x10oyO\xfe\xa0*\x1d=\x14\xd4\xcem\xed.c\xbd\xd0\x04\xb5\x97\t\xdd\xa5\n.%\x01An\x9f\xdb:\xa4;\xfc[\xc3\x0fS\xcb\xa3\x03\xa3\x8e\x00\x88\xd2jc\x90\x88\\\xe2T\x0fx=\x98\xb4<\xb9\x81\x96\xfb?)>k,\xac\x82U4\t\xa0\xee\xb9d$\xcdo\xfc6\x8a*\xfa0X\xa5\xed\x10\x8b\xcc\xee\xe3"\xda\xa3\xf1\x0e\xbd\x13\x05\xefrf.\x89\xee\nD\x11Q7\x02\x0e\r\xcdXY\xf2\xf1`\x8a\xa5]U\x1b\\\x02\xa3\x18\xf7\xa2\xfeQ\xa6V\x02\xc5h\x07\x99\x8d3\x83n^_\xb0\xb7\x19j0kIL\xabYZ@\x08\r&\x9c{\xa5\x9c8\xf7\xf8\xbe\xd0\xc7\x1b\x0b1:P\xec/\xce\xd2T\x19Kehr\xe7Q\xb93>;\xd7\xdc\xa15\xb6k\x8f\xee\x96C\x83\xa8\x1fT\xcb\xbc\xb6\xb1-*\x92\x03\xacF\xd83#\x00\x16\x8b1\xa2\x11]O\x8c\xab\xe6KM\xce\xf9\xef\xb6>tE\xe6\x1b\xb1-\xb5\x1aK55\\\xd42Z\x92\xf4\xc9\x0e\xce\xb9\x01\x8d\xd6N`\xc8\xa0\xba3y\xfb"\x9c\xf0\x7f\xf9\xfe\x0e\xaa\xbd\xe9\x18\xe9\xab\xec#\x07\x92\x96]\xb5U\xc6\xfd\x1d\x1c\xe1\x98\x0cN\xaf\x8fv\xa7\xee\x04\xd5\xbf+\xad\xca\xedd\xb0M\xdcu\xd6\xf3\xb8J\xb3\xd0uy@H\xd6\x10\xa9\xa0\xba\x18\xf2\x84\xac\x8c\xe

Challenge 11

In [20]:
block_size = 16
for i in range(10):
  encrypted_bytes, key, encryption_mode = encryption_oracle('A'*256, block_size)

  target_index = -1
  max_chunk = int(len(encrypted_bytes)/block_size)
    
  for i in range(max_chunk - 1):
    for j in range(i + 1, max_chunk):
      if encrypted_bytes[i*block_size: (i+1)*block_size] == encrypted_bytes[j*block_size: (j+1)*block_size]:
        #print(encrypted_bytes)
        #print(encrypted_bytes[i*block_size: (i+1)*block_size], encrypted_bytes[j*block_size: (j+1)*block_size], i, j)
        target_index = i
        break
    if target_index != -1:
      break
  
  detected_mode = 'ECB' if target_index != -1 else 'CBC'
  print(f'{encryption_mode == detected_mode}, answer: {encryption_mode}, detected: {detected_mode}')

True, answer: CBC, detected: CBC
True, answer: CBC, detected: CBC
True, answer: CBC, detected: CBC
True, answer: CBC, detected: CBC
True, answer: ECB, detected: ECB
True, answer: CBC, detected: CBC
True, answer: CBC, detected: CBC
True, answer: ECB, detected: ECB
True, answer: CBC, detected: CBC
True, answer: CBC, detected: CBC


Challenge 12

In [21]:
block_size = 16
random_key = os.urandom(block_size)
random_bytes = os.urandom(random.randint(5, 10))

encrypted_bytes = new_oracle(plain_text)
print(encrypted_bytes)

# Step 1 Get block_size
#detected_block_size = detect_block_size(plain_text)
#print(f'block size detection:{detected_block_size==block_size} block_size:{block_size} detected_block_size:{detected_block_size}')
# Step 2 Detected ECB mode
detected_mode = detect_ECB_mode(plain_text, block_size)
print(f'detected_mod: {detected_mode}')
# Step 3,4 Craft input
# Step 5,6 match_output
match_output()

TypeError: can only concatenate str (not "bytes") to str

Challenge 13

In [ ]:
block_size = 16
profile = profile_for('foo@bar.com')
print(f'profile: {profile}')
key = random_aes_key()
print(key)
encrypt_profile = encrypt_ECB(str.encode(profile), key, block_size)
print(encrypt_profile)
decrypt_profile = decrypt_ECB(encrypt_profile, key)
print(decrypt_profile)

target = 'email=foo%40bar.com&uid=10&role=admin'
print(f'target: {target}')
encrypt_target = encrypt_ECB(str.encode(target), key, block_size)
print(encrypt_target)
decrypt_target = decrypt_ECB(encrypt_target, key)
print(decrypt_target)

profile: email=foo%40bar.com&uid=10&role=user
b'\xd4V\x962\x18\xe1\xe7\xe6+\x03\xe9_\xdcp\x90\x90'
b'"4\x80\x1fJgH\xda\x92\xd6}:=PO\xae\xc6\xca\xf8|\x98\r\xbcr\x1d\'T\xe3\xf6\x96\xbbD\xec\xe2r\xed\x86\xc1D\xbd\xc7\x94\xca4m\x10\xea\xba'
b'email=foo%40bar.com&uid=10&role=user\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
target: email=foo%40bar.com&uid=10&role=admin
b'"4\x80\x1fJgH\xda\x92\xd6}:=PO\xae\xc6\xca\xf8|\x98\r\xbcr\x1d\'T\xe3\xf6\x96\xbbD\x88\x01=\xea\x05a\xe3Y%\xb8\x0c]\xbc\x12\xd5c'
b'email=foo%40bar.com&uid=10&role=admin\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


In [ ]:
origin = profile_for('foo@bar.com')
print([origin[i*16:(i+1)*16] for i in range(5)])
encrypt_origin = encrypt_ECB(str.encode(origin), key, block_size)
print([encrypt_origin[i*16:(i+1)*16] for i in range(5)])
hack = profile_for('foo@bar.admin')
print([hack[i*16:(i+1)*16] for i in range(5)])
encrypt_hack = encrypt_ECB(str.encode(hack), key, block_size)
print([encrypt_hack[i*16:(i+1)*16] for i in range(5)])

['email=foo%40bar.', 'com&uid=10&role=', 'user', '', '']
[b'"4\x80\x1fJgH\xda\x92\xd6}:=PO\xae', b"\xc6\xca\xf8|\x98\r\xbcr\x1d'T\xe3\xf6\x96\xbbD", b'\xec\xe2r\xed\x86\xc1D\xbd\xc7\x94\xca4m\x10\xea\xba', b'', b'']
['email=foo%40bar.', 'admin&uid=10&rol', 'e=user', '', '']
[b'"4\x80\x1fJgH\xda\x92\xd6}:=PO\xae', b'\x84L\xeaB\xd7\xe3\xf9%W\xab\xb4\x8d\x80\x84\xd6\xf5', b'f[\x0c\n\xbaG\xd7rR\xe0\xbae\xde\x8e\xa1(', b'', b'']


In [ ]:
encrypt_origin = encrypt_origin.replace(encrypt_origin[32:48], encrypt_hack[16:32]) + encrypt_hack[32:48]
print(encrypt_origin)
print(decrypt_ECB(encrypt_origin, key))

b'"4\x80\x1fJgH\xda\x92\xd6}:=PO\xae\xc6\xca\xf8|\x98\r\xbcr\x1d\'T\xe3\xf6\x96\xbbD\x84L\xeaB\xd7\xe3\xf9%W\xab\xb4\x8d\x80\x84\xd6\xf5f[\x0c\n\xbaG\xd7rR\xe0\xbae\xde\x8e\xa1('
b'email=foo%40bar.com&uid=10&role=admin&uid=10&role=user\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


In [ ]:
hack = profile_for('foo@bar.admin')
print([hack[i*16:(i+1)*16] for i in range(5)])
encrypt_hack = encrypt_ECB(str.encode(hack), key, block_size)
print([encrypt_hack[i*16:(i+1)*16] for i in range(5)])

['email=foo%40bar.', 'admin&uid=10&rol', 'e=user', '', '']
[b'"4\x80\x1fJgH\xda\x92\xd6}:=PO\xae', b'\x84L\xeaB\xd7\xe3\xf9%W\xab\xb4\x8d\x80\x84\xd6\xf5', b'f[\x0c\n\xbaG\xd7rR\xe0\xbae\xde\x8e\xa1(', b'', b'']


Challenge 14

In [ ]:
random_prefix = os.urandom(random.randint(1, 32))
print(random_prefix, len(random_prefix))
print(random_key)

b'\x81\xf2\xfd\xd1G\x1c\x1f$\xb9\x991\x83P' 13
b'\x82\xe9Wv\xf1\x93\x04P\xd1^\xd5\x0e\x17C\xd4\x15'


In [ ]:
attacker_controlled = b'attacker_controlled'
encrypted_btyes = new_oracle(random_prefix + attacker_controlled)
print(encrypted_btyes)

b"\x00\xb5\xa4\xe6\xa4\x0b\xefN\xbc'\x1c=N6\x83\xcc p\xc6z\xba\x14\twD\x00\x12\x9dD\xfd\x99\xd3\xaf\r\x1f\x1c|\x91\x02X\xb4fm\xa3\xf3\x10\xe9\x83_p\xa6\xcb\x8c\xb9\xe3\xb6c\x8e\xee\x11\xb2\x90\x19\xa4\x8c\xb0\x8fK\xae\xbf\xd8\xf1U\xe9p\x8b\xd8\xc5z\xd6%e\x06\r\xe9\xd3\xc1\x9d\xa3\x06\x80]\xfc\x18\x9d!\x90\x1aj\x05v\x0cW\x1e\x07\x9c*\xad\xc2\x95\xdcc\xa1\x98@*\x19\xb8\xd2\xe8\xf3\x0e\x89#X\x93\xcbhw\xe1\xed\x1f\xce[K\x905\xfc\xeeECK\xddx^\xc8\xce\xc4\xad\x9d\x9fg=\x1dC\xf5\x9f\xf4\x156\x11r-=\x1a\xa7C\xee%\xb8\x14\x1bU#E\x07\xf4\xe4if^66\x7f\xab\xdf\xbd\x14\xb0\x9f\xa2\x16"


In [ ]:
match_output_14()

0 [b'\xbb>)\xc4\xeee\x05\x12W\x0cT\xde\x05\xe0\xb9\x9a', b'_p\xa6\xcb\x8c\xb9\xe3\xb6c\x8e\xee\x11\xb2\x90\x19\xa4', b'\x8c\xb0\x8fK\xae\xbf\xd8\xf1U\xe9p\x8b\xd8\xc5z\xd6', b'%e\x06\r\xe9\xd3\xc1\x9d\xa3\x06\x80]\xfc\x18\x9d!', b'\x90\x1aj\x05v\x0cW\x1e\x07\x9c*\xad\xc2\x95\xdcc', b'\xa1\x98@*\x19\xb8\xd2\xe8\xf3\x0e\x89#X\x93\xcbh', b'w\xe1\xed\x1f\xce[K\x905\xfc\xeeECK\xddx', b'^\xc8\xce\xc4\xad\x9d\x9fg=\x1dC\xf5\x9f\xf4\x156', b'\x11r-=\x1a\xa7C\xee%\xb8\x14\x1bU#E\x07', b'\xf4\xe4if^66\x7f\xab\xdf\xbd\x14\xb0\x9f\xa2\x16', b'']
1 [b'vGx\xe6\xe7W^b\xe7Oq\xc6R\xfd\xf5\xef', b'\x00?,\x89o.\xa8\x0fx\x18\xf6\xa6\xf0\xc9\x07\xd8', b'6Xu\xa4\xdaj\xc9;\x99U\xd8\xcacEWI', b'\xad\xa9\xbf`l\x06\x86\x9b\xc4\x1fRI}\x08\xbe,', b'<\x84\xe4\x1d\x9a\x1c\xd8i\xa7}<A\x93\r\xd9V', b"\xc7'\x92\xc8\xfd\x15_\x064s\xfd\xb1m.\x0c\x99", b'\x0b\xfcZ\x1d\xf7\xb1^\x93BaU<`\xbc<\xd3', b'\xf1T\xf0\xbc\xfa\xfb\x7f\x96\xf4\xfa\xbb&\xf1\x9c\xe94', b'\x1e\xe0\xa1`\xcc\xff5\x80\n\xe8X/3$M\x98', b'9v\xde\xf4\xa5\x9e

b"Rollin' in my 5.0\nWith my rag-top down so my hair can blow\nThe girlies on standby waving just to say hi\nDid you stop? No, I just drove by\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"

Challenge 15

In [ ]:
plain_bytes = b"ICE ICE BABY\x04\x04\x04\x04"
print(check_and_strip_PKCS7(plain_bytes, block_size))
plain_bytes = b"ICE ICE BABY\x05\x05\x05\x05"
# check_and_strip_PKCS7(plain_bytes, block_size)

b'ICE ICE BABY'


Challenge 16

In [ ]:
block_size = 16
ramdom_key = random_aes_key()
initialization_vector = random_aes_key()
plain_text = "ABCDEFGHIJKLMNOP" * 2

encrypt_bytes = encrypt_CBC_16(plain_text, random_key, block_size, initialization_vector)
print_bytes(encrypt_bytes)
print(check_admin_16(encrypt_bytes, random_key, block_size, initialization_vector))

target_chunk = encrypt_bytes[2*block_size:3*block_size]
target_chunk = [b1 ^ b2 for b1, b2 in zip(target_chunk, b';admin=true;AAAA')]
target_chunk = bytes([b1 ^ b2 for b1, b2 in zip(target_chunk, b"ABCDEFGHIJKLMNOP")])

print(check_admin_16(encrypt_bytes[:2*block_size] + target_chunk + encrypt_bytes[3*block_size:], random_key, block_size, initialization_vector))

[b'\x0f\\\x0bNjlU\xd5\x0c\xfc;\xdd,\xc8\xa8:', b'\x03@\xaa\x89Lyv\x94\xa4[\xfe\xb8\xeby\xd6\xf3', b'\xe5!5\x1fW\x95\xdc#\xcf\x89\x8b\xc7<\xce\xb8j', b'8~\xe9\xd9\x18\xda\xcd\xbf\xff\xd5\xff\xeb\x04s\xde\x97', b'\xc8\x1f\x84\xf7\xd9\x00\xf6\x9fz\xd2\x98@\x81\x16d[', b'\xf0"\xf3\x93\xa2\xfcP\xacM\xcc\x1cS\xec}\xaa\xbd', b'TN\xc4\x17P)\xd6\xd3\xf4\x8a87\xfa,\xeb}']
[b'comment1=cooking', b'%20MCs;userdata=', b'ABCDEFGHIJKLMNOP', b'ABCDEFGHIJKLMNOP', b';comment2=%20lik', b'e%20a%20pound%20', b'of%20bacon\x04\x04\x04\x04\x04\x04']
comment1=cooking%20MCs;userdata=ABCDEFGHIJKLMNOPABCDEFGHIJKLMNOP;comment2=%20like%20a%20pound%20of%20bacon
False
[b'comment1=cooking', b'%20MCs;userdata=', b'\xa2\xca\x01\xfdX!\x1cN\xe7\xc8hh\xa1\xe2\xc3\xb7', b';admin=true;AAAA', b';comment2=%20lik', b'e%20a%20pound%20', b'of%20bacon\x04\x04\x04\x04\x04\x04']
comment1=cooking%20MCs;userdata=¢ÊýX!NçÈhh¡âÃ·;admin=true;AAAA;comment2=%20like%20a%20pound%20of%20bacon
True


Challenge 17

In [103]:
random_string_list = [b'MDAwMDAwTm93IHRoYXQgdGhlIHBhcnR5IGlzIGp1bXBpbmc=',
b'MDAwMDAxV2l0aCB0aGUgYmFzcyBraWNrZWQgaW4gYW5kIHRoZSBWZWdhJ3MgYXJlIHB1bXBpbic=',
b'MDAwMDAyUXVpY2sgdG8gdGhlIHBvaW50LCB0byB0aGUgcG9pbnQsIG5vIGZha2luZw==',
b'MDAwMDAzQ29va2luZyBNQydzIGxpa2UgYSBwb3VuZCBvZiBiYWNvbg==',
b'MDAwMDA0QnVybmluZyAnZW0sIGlmIHlvdSBhaW4ndCBxdWljayBhbmQgbmltYmxl',
b'MDAwMDA1SSBnbyBjcmF6eSB3aGVuIEkgaGVhciBhIGN5bWJhbA==',
b'MDAwMDA2QW5kIGEgaGlnaCBoYXQgd2l0aCBhIHNvdXBlZCB1cCB0ZW1wbw==',
b'MDAwMDA3SSdtIG9uIGEgcm9sbCwgaXQncyB0aW1lIHRvIGdvIHNvbG8=',
b'MDAwMDA4b2xsaW4nIGluIG15IGZpdmUgcG9pbnQgb2g=',
b'MDAwMDA5aXRoIG15IHJhZy10b3AgZG93biBzbyBteSBoYWlyIGNhbiBibG93']

key = random_aes_key()
IV = random_aes_key()
block_size = 16

In [196]:
# function 1
def encrypt_CBC_17(IV: bytes) -> bytes:
    cipher_text = random.choice(random_string_list)
    encrypted_text = encrypt_CBC(cipher_text, key, block_size, IV)
    return encrypted_text

# function 2
def check_padding_is_valid(encrypt_bytes: bytes, IV: bytes) -> bool:
    decrypted_bytes = decrypt_CBC(encrypted_bytes, key, block_size, IV, False)
    target_bytes = decrypted_bytes[-1]
    print(f'decrypted_bytes {decrypted_bytes}')
    print(f'target_bytes {target_bytes}')
    if decrypted_bytes[-target_bytes:] == decrypted_bytes[-1:] * target_bytes:
        return True
    return False

In [197]:
encrypt_bytes = encrypt_CBC_17(IV)
check_padding_is_valid(encrypt_bytes, IV)

NameError: name 'encrypted_bytes' is not defined

In [187]:
for i in range(1, 256):
    new_iv = b'0' * (block_size - 1) + i.to_bytes(1, 'big')
    print(response_17(key, block_size, new_iv))

TypeError: check_padding_is_valid() missing 3 required positional arguments: 'key', 'block_size', and 'IV'